## Import

In [12]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from IPython.display import display, HTML
import sys
sys.path.append("../src")
from vibe_engine import VibeEngine

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data

In [7]:
csv_path = "../data/spotify_tracks.csv"

df = pd.read_csv(csv_path)
df_tracks = df.dropna().reset_index(drop=True)
del df_tracks[df_tracks.columns[0]]
df_tracks.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


## Initialization

In [4]:
engine = VibeEngine(df_tracks)

print("VibeEngine initialized.")
print("Num tracks:", engine.X.shape[0])
print("Num vibe dims:", engine.X.shape[1])
print("Vibe cols:", engine.vibe_cols)
print("Has track_id mapping:", len(engine.trackid_to_idx) > 0)

VibeEngine initialized.
Num tracks: 114000
Num vibe dims: 7
Vibe cols: ['danceability', 'energy', 'valence', 'tempo', 'acousticness', 'instrumentalness', 'speechiness']
Has track_id mapping: True


## Mode 1

In [8]:
from mode1_sliders import SliderVibeRecommender

mode1 = SliderVibeRecommender(df_tracks)

sliders = {
    "danceability": 80,
    "energy": 75,
    "valence": 70,
    "tempo": 65,
    "acousticness": 20,
    "instrumentalness": 10,
    "speechiness": 15,
}

recs_mode1 = mode1.recommend_by_sliders(
    sliders,
    top_k=10,
    lambda_vibe=0.8,
    hide_explicit=True,
)

recs_mode1[["track_name", "artists", "track_genre", "vibe_score", "vibe_similarity"]].head(10)

,track_name,artists,track_genre,vibe_score,vibe_similarity
89912,Un Coco,Bad Bunny,reggaeton,0.939900,0.957376
89771,Sobrio,Maluma,reggaeton,0.934318,0.965397
3255,Feel Good Inc.,Gorillaz,alternative,0.929595,0.951994
89819,Se Preparó,Ozuna,reggaeton,0.919002,0.961253
81173,I Ain't Worried,OneRepublic,pop,0.907499,0.894374
20407,Right Now (Na Na Na),Akon,dance,0.899292,0.946615
51122,Satisfya,Imran Khan,hip-hop,0.898673,0.948341
104802,No Me Basta,India Martinez;Dvicio,spanish,0.896389,0.975486
39408,Bamba (feat. Aitch & BIA),Luciano;Aitch;BIA,german,0.888171,0.900214
65109,Black Swan,BTS,k-pop,0.881009,0.916262


In [14]:
def show_vibe_recs(model, sliders, top_k=10, **kwargs):
    # 1. Get recommendations
    recs = model.recommend_by_sliders(
        sliders,
        top_k=top_k,
        **kwargs,
    )

    if recs.empty:
        print("No recommendations found for this vibe 😢 Try relaxing your filters.")
        return recs
    
def describe_vibe(sliders):
    d = sliders["danceability"]
    e = sliders["energy"]
    v = sliders["valence"]

    mood_bits = []
    if e > 70: mood_bits.append("high energy")
    elif e < 35: mood_bits.append("low energy")

    if d > 70: mood_bits.append("very danceable")
    elif d < 35: mood_bits.append("chill / not very danceable")

    if v > 65: mood_bits.append("generally happy")
    elif v < 35: mood_bits.append("on the sadder side")

    return " • ".join(mood_bits) or "balanced vibe"

In [15]:
def show_now_playing_and_queue(recs: pd.DataFrame):
    if recs.empty:
        print("No recommendations returned 🙁")
        return

    # row 0 = Now Playing
    now = recs.iloc[0]
    print("🎧 NOW PLAYING")
    print(f"  {now['track_name']} — {now['artists']}")
    print(f"  Genre: {now.get('track_genre', 'N/A')}")
    if "vibe_similarity" in now:
        print(f"   Vibe match: {now['vibe_similarity']:.3f} | Score: {now['vibe_score']:.3f}")
    if "track_id" in now and isinstance(now["track_id"], str):
        url = f"https://open.spotify.com/track/{now['track_id']}"
        display(HTML(f'<p><a href="{url}" target="_blank">▶ Open in Spotify</a></p>'))

    # rows 1–9 = Up Next
    if len(recs) > 1:
        up_next = recs.iloc[1:10][["track_name", "artists", "track_genre", "vibe_score"]]
        print("⬇️ UP NEXT")
        display(up_next.reset_index(drop=True))
    else:
        print("No additional tracks in queue.")

print("Vibe summary:", describe_vibe(sliders))
show_now_playing_and_queue(recs_mode1)

Vibe summary: high energy • very danceable • generally happy
🎧 NOW PLAYING
  Un Coco — Bad Bunny
  Genre: reggaeton
   Vibe match: 0.957 | Score: 0.940


⬇️ UP NEXT


,track_name,artists,track_genre,vibe_score
0,Sobrio,Maluma,reggaeton,0.934318
1,Feel Good Inc.,Gorillaz,alternative,0.929595
2,Se Preparó,Ozuna,reggaeton,0.919002
3,I Ain't Worried,OneRepublic,pop,0.907499
4,Right Now (Na Na Na),Akon,dance,0.899292
5,Satisfya,Imran Khan,hip-hop,0.898673
6,No Me Basta,India Martinez;Dvicio,spanish,0.896389
7,Bamba (feat. Aitch & BIA),Luciano;Aitch;BIA,german,0.888171
8,Black Swan,BTS,k-pop,0.881009


## Mode 2A

In [20]:
from mode2a_seed_from_song import Mode2ASeedFromSong

mode2a = Mode2ASeedFromSong(engine)
mode2a.df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [21]:
query = "feel good inc"  # or "bad bunny", "taylor swift", etc.

candidates = mode2a.search_tracks(query, max_results=10)
display(candidates[["track_name", "artists", "track_genre", "track_id"]])

,track_name,artists,track_genre,track_id
3255,Feel Good Inc.,Gorillaz,alternative,0d28khcov6AiegSCpG5TuT


### Use `df_index` as seed

In [27]:
seed_row = candidates.iloc[0]
seed_track_id = seed_row["track_id"]

print("Using this track_id as seed:", seed_track_id)
print(seed_row[["track_name", "artists", "track_genre"]])

seed_url = f"https://open.spotify.com/track/{seed_track_id}"
print("🔗 Open SEED in Spotify:", seed_url)

now_playing2, up_next2, seed_idx2 = mode2a.recommend_from_seed(
    track_id=seed_track_id,
    top_k=10,
    lambda_vibe=0.8,
    hide_explicit=True,
)

mode2a.print_now_playing_and_queue(
    now_playing=now_playing2,
    up_next=up_next2,
    seed_idx=seed_idx2,
    show_explanation=True,
    show_spotify_embed=True,
)


Using this track_id as seed: 0d28khcov6AiegSCpG5TuT
track_name     Feel Good Inc.
artists              Gorillaz
track_genre       alternative
Name: 3255, dtype: object
🔗 Open SEED in Spotify: https://open.spotify.com/track/0d28khcov6AiegSCpG5TuT
🎧 Now playing: Ferrari — James Hype;Miggy Dela Rosa
▶ Open in Spotify: https://open.spotify.com/track/4zN21mbAuaD0WqtmaTZZeP



Recommended **Ferrari** by James Hype;Miggy Dela Rosa because its vibe closely matches *Feel Good Inc.* on: instrumental vibe 🎼, acoustic feel 🎸, energy ⚡.

🎵 Up Next:


,track_name,artists,track_genre,vibe_score,vibe_similarity
79000,I Ain't Worried,OneRepublic,piano,0.920598,0.910747
89913,Un Coco,Bad Bunny,reggaeton,0.918057,0.930071
31014,One Kiss (with Dua Lipa),Calvin Harris;Dua Lipa,electro,0.916758,0.923447
39408,Bamba (feat. Aitch & BIA),Luciano;Aitch;BIA,german,0.916688,0.935860
91108,Stressed Out,Twenty One Pilots,rock,0.904524,0.918155
86105,Should I Stay or Should I Go - Remastered,The Clash,punk,0.901864,0.927330
65473,Savage,aespa,k-pop,0.896501,0.935627
53270,Personal,HRVY,house,0.896305,0.945381
1500,La Vuelta Al Mundo,Calle 13,afrobeat,0.894197,0.942746


### Use `track_id` as seed

In [28]:
seed_row = candidates.iloc[0]
seed_track_id = seed_row["track_id"]

print("Using this track_id as seed:", seed_track_id)
print(seed_row[["track_name", "artists", "track_genre"]])

now_playing2, up_next2, seed_idx2 = mode2a.recommend_from_seed(
    track_id=seed_track_id,
    top_k=10,
    lambda_vibe=0.8,
    hide_explicit=True,
)

mode2a.print_now_playing_and_queue(
    now_playing=now_playing2,
    up_next=up_next2,
    seed_idx=seed_idx2,
    show_explanation=True,
)


Using this track_id as seed: 0d28khcov6AiegSCpG5TuT
track_name     Feel Good Inc.
artists              Gorillaz
track_genre       alternative
Name: 3255, dtype: object
🎧 Now playing: Ferrari — James Hype;Miggy Dela Rosa
▶ Open in Spotify: https://open.spotify.com/track/4zN21mbAuaD0WqtmaTZZeP



Recommended **Ferrari** by James Hype;Miggy Dela Rosa because its vibe closely matches *Feel Good Inc.* on: instrumental vibe 🎼, acoustic feel 🎸, energy ⚡.

🎵 Up Next:


,track_name,artists,track_genre,vibe_score,vibe_similarity
79000,I Ain't Worried,OneRepublic,piano,0.920598,0.910747
89913,Un Coco,Bad Bunny,reggaeton,0.918057,0.930071
31014,One Kiss (with Dua Lipa),Calvin Harris;Dua Lipa,electro,0.916758,0.923447
39408,Bamba (feat. Aitch & BIA),Luciano;Aitch;BIA,german,0.916688,0.935860
91108,Stressed Out,Twenty One Pilots,rock,0.904524,0.918155
86105,Should I Stay or Should I Go - Remastered,The Clash,punk,0.901864,0.927330
65473,Savage,aespa,k-pop,0.896501,0.935627
53270,Personal,HRVY,house,0.896305,0.945381
1500,La Vuelta Al Mundo,Calle 13,afrobeat,0.894197,0.942746
